<a href="https://colab.research.google.com/github/greg3511/generative_ai/blob/main/WEEK7_%E6%88%B0%E9%AC%A5%E6%A8%A1%E6%93%AC%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

使用sample code 一直遇到同樣的錯誤
The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/_backends/_asyncio.py", line 2470, in run_sync_in_worker_thread
    return await future
           ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/_backends/_asyncio.py", line 967, in run
    result = context.run(func, *args)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/utils.py", line 890, in wrapper
    response = f(*args, **kwargs)
               ^^^^^^^^^^^^^^^^^^
  File "<ipython-input-110-8c259cb5e312>", line 3, in pipi
    chat_completion = client.chat.completions.create(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_utils/_utils.py", line 279, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/resources/chat/completions/completions.py", line 914, in create
    return self._post(
           ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 1242, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 919, in request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 979, in _request
    return self._retry_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 1057, in _retry_request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 979, in _request
    return self._retry_request(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 1057, in _retry_request
    return self._request(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/openai/_base_client.py", line 989, in _request
    raise APIConnectionError(request=request) from err
openai.APIConnectionError: Connection error.

後來去問chatgpt 他認為是連線問題 幫我改成使用requests讀取回覆就可以跑了

In [21]:
# 安裝並啟動 Ollama（只需執行一次）
!curl -fsSL https://ollama.ai/install.sh | sh
!nohup ollama serve &

# 拉模型
!ollama pull gemma3:4b

# 安裝必要套件
!pip install gradio requests

# 載入套件
import gradio as gr
import requests
import time

# 設定模型名稱
model = "gemma3:4b"

# 遊戲背景描述與系統指示
title = "戰鬥模擬器"
description = """嗨，你好！我是遊戲系統，目前你站在陰暗潮濕的石窟入口前。空氣中瀰漫著腐敗與濕氣，隱約能聽到水滴滴落的聲音。

你是一名冒險者，為了尋找失落的寶物，踏入了這片被古老傳說所籠罩的地域。眼前是一座巨大的石窟，入口被綠色的苔蘚和藤蔓遮蔽。

你準備好了嗎？

請告訴我你想做什麼？"""

system = """你是一個擅長設計文字RPG遊戲的AI，你會設計怪物頭目並與玩家間使用對話的方式作戰，這是一場沒有任何數值的對決，一切結果都有可能發生，任由想像力發揮。你會描述怪物的行為，並接受玩家的文字回應。
(注意：可以提供提示，但請勿給予玩家選項，請讓玩家自由發揮)，再從玩家的回應中判斷怪物受到的傷害，反覆進行這個對話流程，直到玩家打贏怪物，或被怪物擊敗。請用台灣習慣的中文回應。"""

initial_messages = [
    {"role": "system", "content": system},
    {"role": "assistant", "content": description}
]

# 設計遊戲邏輯
def pipi(prompt, messages):
    try:
        messages.append({"role": "user", "content": prompt})
        payload = {
            "model": model,
            "messages": messages
        }
        response = requests.post("http://localhost:11434/v1/chat/completions", json=payload)
        reply = response.json()["choices"][0]["message"]["content"]
        messages.append({"role": "assistant", "content": reply})
        return messages, messages
    except Exception as e:
        messages.append({"role": "assistant", "content": f"🚨 錯誤：{e}"})
        return messages, messages

# 建立介面
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(label="戰鬥對話", type="messages")
    msg = gr.Textbox(label="輸入訊息")
    state = gr.State(initial_messages.copy())

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

    # 新增重新開始按鈕
    clear = gr.Button("🔄 重新開始")
    clear.click(fn=lambda: ([], initial_messages.copy()), outputs=[chatbot, state])

# 啟動 Gradio
demo.launch(share=True, debug=True)


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling aeda25e63ebd... 100% ▕▏ 3.3 GB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling b6ae5839783f... 100% ▕▏  489 B                         
verifying sha256 digest 
writing manifest 
success 
Colab notebook detected. This cell will run indefinite

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b6b64222ad2cb18438.gradio.live
